In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2 as cv
import cv2
import time
import warnings

import tensorflow as tf
from tensorflow.keras.models import load_model
# from sklearn.linear_model.logistic import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from imageio import imread
from skimage.transform import resize

import tensorflow.compat.v1 as tf1

In [2]:
from scipy.spatial import distance
import json
import csv
import pickle
import pandas as pd

In [3]:
# haar_path = "/usr/share/opencv4/haarcascades/haarcascade_frontalface_default.xml"
haar_path = "./models/haarcascade_frontalface_alt2.xml"
train_data_path = './data/train/'
names = []
unknownid = 0
train_folders = os.listdir("./data/train")
for i in train_folders:
    if not os.path.isdir(i):
        names.append(i)
print(names)
for i in names:
    if (i=="0Unknown"):
        break
    unknownid+=1
print(unknownid)
image_size = 160

['Hera Chan', '0Unknown', 'Sharon Chan', 'Nathan', '309', '303', '300', '390', 'Shinji Kagawa', 'Pony Ma', '302', '306', 'MarkZuckerberg', 'Tamara Mowry', 'Jay Chou', 'Shinjiro Koizumi', '395', '304', 'LarryPage', '308', '387', 'JFK', '305', '366', 'Andrew Ng', 'Wong Tsui-yu', '312', 'BillGates', '396', 'YiYang Qianxi', 'Li Yifeng', 'Jackson', '310', 'Leehom Wang', 'vasili-berezutski', '311', 'Wang Junkai']
1


In [4]:
start = time.time()

# model_path = './models/trt/facenet_trt_graph.pb'
# model_path = './facenet/facenet.pb'
# model_path = './TFTRT_16/saved_model.pb'

# config_path = './models/trt/facenet_trt.pbtxt'

# model_path = './models/fn_vgg2_fp16.pb'
# model_path = './pb/fn_vgg2.pb'
# model_path = './pb/fn_vgg2_fp32.pb'
# model_path = './pb/fn_vgg2_fp16_convert.pb'
# model_path = './pb/fn_fp16_1GB.pb'
# model_path = './pb/fn_fp16_2GB.pb'
# model_path = './pb/fn_fp16_256MB.pb'
model_path = './pb/fn_fp16_128MB.pb'
# model_path = './pb/fn_fp16_64MB.pb'

# model_path = './pb/fn_fp16_4GB.pb'
# model_path = './pb/fn_fp16_3GB.pb'
# model_path = './pb/fn_fp16_2g_test.pb'
# model_path = './pb/fn_int8_1G.pb'

with tf.compat.v1.gfile.FastGFile(model_path, 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    em = tf1.import_graph_def(graph_def, input_map = None, name='',return_elements=["Bottleneck_BatchNorm/batchnorm_1/add_1:0"])
#     tf1.import_graph_def(graph_def, input_map = None, name='')
    
end = time.time()
print(end-start)

5.576957941055298


In [5]:
# Get input and output tensors
# images_placeholder = tf1.get_default_graph().get_tensor_by_name("input:0")
images_placeholder = tf1.get_default_graph().get_tensor_by_name("input_1:0")

# embeddings = tf1.get_default_graph().get_tensor_by_name("embeddings:0")
embeddings = tf1.get_default_graph().get_tensor_by_name("Bottleneck_BatchNorm/batchnorm_1/add_1:0")

# phase_train_placeholder = tf1.get_default_graph().get_tensor_by_name("phase_train:0")
embedding_size = embeddings.get_shape()[1]
print(embedding_size)
# print(em.size)

512


In [6]:
def pkl_save(data,path):
    file = open(path,'wb')
    pickle.dump(data, file)
    file.close()

In [7]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

In [8]:
def load_and_align_images(filepaths, margin):
    cascade = cv2.CascadeClassifier(haar_path)
    
    aligned_images = []
    for filepath in filepaths:
        print(filepath)
        img = imread(filepath)

        faces = cascade.detectMultiScale(img,
                                         scaleFactor=1.05,
                                         minNeighbors=5)
        (x, y, w, h) = faces[0]
        cropped = img[y-margin//2:y+h+margin//2,
                      x-margin//2:x+w+margin//2, :]
        aligned = resize(cropped, (image_size, image_size), mode='reflect')
        aligned_images.append(aligned)
            
    return np.array(aligned_images)

In [9]:
def calc_embs(filepaths, name, margin=10, batch_size=1):
    gpu_options = tf1.GPUOptions(per_process_gpu_memory_fraction=0.4, allow_growth=True)
    with tf.compat.v1.Session(config=tf1.ConfigProto(gpu_options=gpu_options, log_device_placement=False)) as sess:
#         if (name!="0Unknown"):
        aligned_images = prewhiten(load_and_align_images(filepaths, margin))
#         if(name=="0Unknown"):
#             aligned_images = load_and_align_images(filepaths, margin)
        pd = []
        for start in range(0, len(aligned_images), batch_size):
#             feed_dict = { images_placeholder:aligned_images[start:start+batch_size], phase_train_placeholder:False }
            feed_dict = { images_placeholder:aligned_images[start:start+batch_size]}
            emb_array = sess.run(embeddings, feed_dict=feed_dict)
            pd.append(emb_array)
    #         pd.append(model.predict_on_batch(aligned_images[start:start+batch_size]))
        embs = l2_normalize(np.concatenate(pd))
    
    return embs

In [10]:
def inf_embs(img, margin=10, batch_size=1):
    aligned_images = prewhiten(img)
    pd = []
#     for start in range(0, 1, batch_size):
#     with tf.compat.v1.Session() as sess:
#     feed_dict = { images_placeholder:aligned_images, phase_train_placeholder:False }
    feed_dict = { images_placeholder:aligned_images}
#     feed_dict = { images_placeholder:img}
    emb_array = sess.run(embeddings, feed_dict=feed_dict)
    pd.append(emb_array)

#     pd.append(model.predict(aligned_images))
    embs = l2_normalize(np.concatenate(pd))
    
    return embs

In [11]:
def calc_dist(img_name0, img_name1):
    return distance.euclidean(data[img_name0]['emb'], data[img_name1]['emb'])

In [12]:
def calc_infer_dist(img_name0, inf_emb):
    return distance.euclidean(data[img_name0]['emb'], inf_emb)

In [13]:
def train_svm(dir_basepath, names, max_num_img=50):
    labels = []
    embs = []
    data = {}
    for name in names:
        dirpath = os.path.abspath(dir_basepath + name)
        filepaths = [os.path.join(dirpath, f) for f in os.listdir(dirpath)][:max_num_img]
        embs_ = calc_embs(filepaths, name)    
        labels.extend([name] * len(embs_))
        embs.append(embs_)
        for i in range(len(filepaths)):
            data['{}{}'.format(name, i)] = {'image_filepath' : filepaths[i],
                                            'emb' : embs_[i]}
        
    embs = np.concatenate(embs)
    le = LabelEncoder().fit(labels)
    y = le.transform(labels)
    pkl_save(embs,'./pkl/embs.pkl')
    pkl_save(y,'./pkl/y.pkl')
    clf = SVC(kernel='linear', probability=True).fit(embs, y)
    return le, clf, data

In [14]:
def train_softmax(dir_basepath, names, max_num_img=10):
    labels = []
    embs = []
    data = {}
    for name in names:
        dirpath = os.path.abspath(dir_basepath + name)
        filepaths = [os.path.join(dirpath, f) for f in os.listdir(dirpath)][:max_num_img]
        embs_ = calc_embs(filepaths, name)    
        labels.extend([name] * len(embs_))
        embs.append(embs_)
        for i in range(len(filepaths)):
            data['{}{}'.format(name, i)] = {'image_filepath' : filepaths[i],
                                            'emb' : embs_[i]}
        
    embs = np.concatenate(embs)
    le = LabelEncoder().fit(labels)
    y = le.transform(labels)
    pkl_save(embs,'./pkl/embs.pkl')
    pkl_save(y,'./pkl/y.pkl')
    clf = LogisticRegression()
    clf.fit(embs, y)
    return le, clf, data

In [15]:
def infer(le, clf, img):
    warnings.filterwarnings(action='ignore', category=DeprecationWarning)
    embs = inf_embs(img)
    clfpred = clf.predict(embs)
    clfprob = clf.predict_proba(embs)
    pred = le.inverse_transform(clfpred)
    if clfprob[0][unknownid]>clfprob[0][clfpred[0]]:
        pred[0]="0Unknown"
#     calc_infer_dist(clfpred[0]+'0', embs[0])

    data_name = pred[0]+'1'
    dist = distance.euclidean(data[data_name]['emb'], embs[0])
    if(dist>0.88):
         pred[0]="0Unknown"
            
    print(pred[0]+"  "+str(dist))
#     print(pred[0]+"  "+str(format(100*clfprob[0][clfpred[0]], '.2f')))
#     print(pred[0]+"  "+str(format(100*clfprob[0][clfpred[0]], '.2f'))+
#           "    "+str(100*clfprob[0][unknownid]))
#     print(clfprob[0][unknownid])
    
    return pred

In [ ]:
start = time.time()
le, clf, data = train_svm(train_data_path, names)
# le, clf = train_softmax(train_data_path, names)
end = time.time()
print(str(end-start))

/home/nathan/lab/fyp/data/train/Hera Chan/5DM36813a.jpg
/home/nathan/lab/fyp/data/train/Hera Chan/Hera Chan3
/home/nathan/lab/fyp/data/train/Hera Chan/40042181_250036812510987_7588053704075378688_n.jpg
/home/nathan/lab/fyp/data/train/Hera Chan/Hera Chan2.jpg
/home/nathan/lab/fyp/data/train/Hera Chan/Hera Chan1.jpg


In [23]:
start = time.time()
# with open('./json/data.json','w') as data_json:
#     json.dump(data, data_json)
# with open('./json/le.json','w') as le_json:
#     json.dump(le, le_json)
# with open('./json/clf.json','w') as clf_json:
#     json.dump(clf, clf_json)
# # end = time.time()

# with open('./pkl/data.csv','w')as data_file:
#     writer = csv.writer(data_file)
#     writer.writerows(data)

data_file = open('./pkl/data.pkl','wb')
pickle.dump(data, data_file)
data_file.close()

le_file = open('./pkl/le.pkl','wb')
pickle.dump(le, le_file)
le_file.close()

# y_file = open('./pkl/clf.pkl','wb')
# pickle.dump(le, clf_file)
# clf_file.close()
    
print(str(time.time()-start))

0.01046133041381836


In [16]:
start = time.time()

data_file = open('./pkl/data.pkl','rb')
data = pickle.load(data_file)

le_file = open('./pkl/le.pkl','rb')
le = pickle.load(le_file)

y_file = open('./pkl/y.pkl','rb')
y = pickle.load(y_file)

embs_file = open('./pkl/embs.pkl','rb')
embs = pickle.load(embs_file)
clf = SVC(kernel='linear', probability=True).fit(embs, y)

print(str(time.time()-start))

0.6096353530883789


In [19]:
print(model_path)

import cv2
import time

cam_height = 480
cam_width = 640

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, cam_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cam_height)
face_cascade = cv2.CascadeClassifier(
    "/usr/share/opencv4/haarcascades/haarcascade_frontalface_default.xml"
)

margin = 10

rec = []
temprec = []
showtext = "Last attendance taken:"
record_dir = "./record/"+time.strftime("%Y-%m-%d",time.localtime())+".csv"

# print(str(time.time()-qdtime))

gpu_options = tf1.GPUOptions(per_process_gpu_memory_fraction=0.6, allow_growth=True)
# gpu_options = tf1.GPUOptions(allow_growth=True)

with tf.compat.v1.Session(config=tf1.ConfigProto(gpu_options=gpu_options, log_device_placement=False)) as sess:
    
    
    while(True):
        start = time.time()

        ret, frame = cap.read()
    #     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1,minNeighbors=12, minSize=(210,210))

        aligned_images = []

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cropped = frame[y-margin//2:y+h+margin//2, x-margin//2:x+w+margin//2, :]
            
            if cropped.size>0:
                aligned = resize(cropped, (image_size, image_size), mode='reflect')
                aligned_images.append(aligned)
                al=np.array(aligned_images)
                pred = infer(le, clf, al)
                cv2.putText(frame, pred[0], (x, y+h+55), cv2.FONT_HERSHEY_DUPLEX, 2, (255, 255, 0), 1, cv2.LINE_AA)
                temprec.append(pred[0])
        if(len(temprec)>1) and (temprec[0]!=temprec[1]):
            temprec=[]
#         if(len(temprec)>2):
#             if(temprec[1]!=temprec[2]):
#                 temprec=[]
#             if(len(temprec)>3):
#                 if(temprec[0]!='0Unknown') and (temprec[2]==temprec[3]):
#     #                 if(temprec[0]==temprec[1]) and (temprec[1]==temprec[2]):
#                     rec.append([temprec[0],time.strftime("%H:%M:%S",time.localtime())])
#                 temprec=[]
        if(len(temprec)>2):
            if(temprec[0]!='0Unknown') and (temprec[1]==temprec[2]):
    #                 if(temprec[0]==temprec[1]) and (temprec[1]==temprec[2]):
                rec.append([temprec[0],time.strftime("%H:%M:%S",time.localtime())])
            temprec=[]
        
        end = time.time()
        t = end-start
        fps="FPS: "+ str(format(1/t, '.2f'))
        cv2.putText(frame,fps, (30, cam_height-25), cv2.FONT_HERSHEY_DUPLEX, 1, (80, 250, 0), 1, cv2.LINE_AA)
        cv2.putText(frame,showtext, (6, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 20, 240), 1, cv2.LINE_AA)
        reclen = len(rec)
        if(reclen>0):
            cv2.putText(frame,rec[reclen-1][0], (5, 67), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 240, 240), 1, cv2.LINE_AA)

        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            with open(record_dir, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(rec)
            break
        
        
cap.release()
cv2.destroyAllWindows()

./pb/fn_fp16_128MB.pb
Nathan  0.7902138829231262
Nathan  0.8226715922355652
Nathan  0.7668718695640564
Nathan  0.781987190246582
Nathan  0.7522034645080566
Nathan  0.8023499250411987
Nathan  0.8194360136985779
Nathan  0.7896414995193481
Nathan  0.8501636385917664
Nathan  0.8190929889678955
Nathan  0.8024623394012451
Nathan  0.7693341374397278
Nathan  0.8043242692947388
Nathan  0.7870920896530151
Nathan  0.8598878979682922
Nathan  0.7856361269950867
Nathan  0.7965894341468811
Nathan  0.7827425599098206
Nathan  0.7843697667121887
Nathan  0.7556235790252686
Nathan  0.7455965876579285
Nathan  0.7451122999191284
Nathan  0.7618705630302429
Nathan  0.7338384389877319
Nathan  0.6855798363685608
Nathan  0.7344627976417542
Nathan  0.7099981307983398
Nathan  0.8066346645355225
Nathan  0.7893074154853821
Nathan  0.7811490297317505
Nathan  0.7623578310012817
Nathan  0.762244462966919
Nathan  0.751873791217804
Nathan  0.7149270176887512
Nathan  0.8090351819992065
Nathan  0.7718557715415955
Nathan  0

In [19]:
print(time.strftime("%H:%M:%S",time.localtime()))
print(rec)

12:20:45
[]


In [6]:
record_dir = "./record/"+time.strftime("%Y-%m-%d",time.localtime())+".csv"
print(record_dir)

./record/2020-11-15.csv


In [75]:
calc_dist('Nathan1', '0Unknown6')

1.3910821676254272

In [18]:
print(data['Nathan1']['emb'])

[-0.02374891 -0.16279574 -0.18202925  0.0132421   0.05303483 -0.10639483
 -0.02464002  0.03658819  0.05894117 -0.00830569 -0.08940133 -0.08434287
 -0.10085908 -0.17689686 -0.02798717  0.16431892  0.02811297  0.0163331
  0.0135625  -0.1377588   0.0758251  -0.02385496 -0.11292622  0.13027951
  0.00181397  0.04516483  0.06570624  0.07880565  0.07351717  0.08587142
  0.06252653  0.1511896   0.01714985 -0.10601576  0.03743646 -0.0259266
 -0.15917556  0.0364768   0.1411299   0.05691886 -0.03003151  0.0105058
  0.00331367 -0.22300385  0.03410346  0.05233781  0.08406793  0.00426664
 -0.15825085  0.03290462 -0.10899365  0.09360295  0.03818059  0.1291093
  0.08436608  0.07097912 -0.09959361 -0.02395344  0.08700064  0.02854469
 -0.05791701 -0.15454358  0.03579155  0.04000003  0.13674608  0.07451902
  0.13111219  0.19054182 -0.0162045  -0.04851913  0.01898834 -0.01989911
 -0.01311679  0.01555526 -0.04555842  0.00229102 -0.04842564  0.1145331
 -0.08079828 -0.11650965 -0.06864829 -0.07694007 -0.0177

In [ ]:
0Unknown  1.2032731771469116
0Unknown  1.1856186389923096
0Unknown  1.1704256534576416
0Unknown  1.1633615493774414
0Unknown  1.0508038997650146
0Unknown  1.100322961807251
0Unknown  1.0065808296203613
0Unknown  0.9992518424987793
0Unknown  1.127713680267334
0Unknown  1.161186695098877
0Unknown  1.1859182119369507
0Unknown  1.1681596040725708
0Unknown  1.1454689502716064
0Unknown  1.1477329730987549
0Unknown  1.200191617012024
0Unknown  1.1773043870925903
0Unknown  1.1665984392166138
0Unknown  1.176975131034851
0Unknown  1.05929696559906
312  0.8020032048225403
0Unknown  1.3566821813583374
0Unknown  1.403282642364502
0Unknown  1.0196784734725952
0Unknown  1.5064764022827148
0Unknown  1.1369071006774902
0Unknown  0.8867366909980774
Li Yifeng  0.7876330018043518
Li Yifeng  0.7437747716903687
Li Yifeng  0.702875018119812
Li Yifeng  0.7246072292327881
Li Yifeng  0.7429304122924805
Li Yifeng  0.8098421096801758
Li Yifeng  0.7990174889564514
Li Yifeng  0.8216648697853088
Li Yifeng  0.7907418608665466
Li Yifeng  0.7766798138618469
Li Yifeng  0.7990901470184326
Li Yifeng  0.7447104454040527
Li Yifeng  0.8025739789009094
Li Yifeng  0.799933135509491
Li Yifeng  0.8400545120239258
0Unknown  1.2348089218139648
Leehom Wang  0.7534019351005554
Leehom Wang  0.6190147995948792
Leehom Wang  0.6399226784706116
Leehom Wang  0.6223984360694885
Leehom Wang  0.6166989207267761
Leehom Wang  0.618927001953125
Leehom Wang  0.6025663614273071
Leehom Wang  0.6499391198158264
Leehom Wang  0.6367903351783752
Leehom Wang  0.6657533049583435
Leehom Wang  0.6249403953552246
Leehom Wang  0.5942284464836121
Leehom Wang  0.5538001656532288
Leehom Wang  0.6024282574653625
Leehom Wang  0.5666455030441284
Leehom Wang  0.641990065574646
Leehom Wang  0.63669353723526
Leehom Wang  0.6272927522659302
Leehom Wang  0.6191508173942566
Leehom Wang  0.6028557419776917
0Unknown  1.200416088104248
0Unknown  1.1929219961166382
0Unknown  1.263641595840454
0Unknown  1.3098535537719727
0Unknown  1.2721434831619263
0Unknown  1.1771951913833618
0Unknown  1.1762996912002563
0Unknown  1.2215474843978882
0Unknown  0.9222264885902405
0Unknown  0.8845903277397156
0Unknown  0.9056447744369507
0Unknown  0.9068951606750488
0Unknown  0.9185298681259155
0Unknown  0.9007555842399597
0Unknown  0.9214547872543335
0Unknown  1.3303683996200562
0Unknown  1.3917315006256104
0Unknown  1.3156710863113403
0Unknown  1.31962251663208
0Unknown  1.3422040939331055
0Unknown  1.3451433181762695
0Unknown  1.2876254320144653
0Unknown  1.1006933450698853
0Unknown  1.1154605150222778
0Unknown  1.098645567893982
0Unknown  1.0947282314300537
0Unknown  1.0757684707641602
0Unknown  1.0911893844604492
0Unknown  1.1013593673706055
0Unknown  1.1179171800613403
0Unknown  1.1398802995681763
0Unknown  1.09486985206604
0Unknown  1.1148005723953247
0Unknown  1.1117326021194458
0Unknown  1.1452584266662598
0Unknown  1.0889325141906738
0Unknown  1.099131464958191
0Unknown  1.1053006649017334
0Unknown  1.126327633857727
0Unknown  1.1164675951004028
0Unknown  1.1060411930084229
0Unknown  1.1298651695251465
0Unknown  1.1248329877853394
0Unknown  1.1408532857894897
0Unknown  1.133594036102295
0Unknown  1.1480085849761963
0Unknown  1.1356736421585083
0Unknown  1.1872318983078003
0Unknown  1.0106751918792725
Sharon Chan  0.8230265378952026
0Unknown  1.2439613342285156
Sharon Chan  0.8018079400062561
Sharon Chan  0.7281743884086609
Sharon Chan  0.7134149670600891
Sharon Chan  0.6916524767875671
Sharon Chan  0.7565081715583801
Sharon Chan  0.6886011958122253
Sharon Chan  0.6732286810874939
Sharon Chan  0.7310529351234436
Sharon Chan  0.6827953457832336
Sharon Chan  0.678891122341156
Sharon Chan  0.7170721292495728
Sharon Chan  0.6940332651138306
Sharon Chan  0.7305235862731934
Sharon Chan  0.6873879432678223
Sharon Chan  0.7501769065856934
Sharon Chan  0.7467215657234192
Sharon Chan  0.6584348678588867
Sharon Chan  0.6636425852775574
Sharon Chan  0.6480562090873718
Sharon Chan  0.6193265914916992
Sharon Chan  0.5764910578727722
Sharon Chan  0.6641165018081665
Sharon Chan  0.6659148931503296
Sharon Chan  0.6976224184036255
0Unknown  0.9050646424293518
Shinji Kagawa  0.650076687335968
Shinji Kagawa  0.6332035660743713
Shinji Kagawa  0.5765713453292847
Shinji Kagawa  0.5232950448989868
Shinji Kagawa  0.5898124575614929
Shinji Kagawa  0.6090708374977112
Shinji Kagawa  0.4966305196285248
Shinji Kagawa  0.5121342539787292
Shinji Kagawa  0.5422458648681641
Shinji Kagawa  0.5375446081161499
Shinji Kagawa  0.5509054064750671
Shinji Kagawa  0.526299774646759
Shinji Kagawa  0.527497410774231
Shinji Kagawa  0.5420034527778625
Shinji Kagawa  0.5337199568748474
Shinji Kagawa  0.5082389116287231
Shinji Kagawa  0.5428707599639893
Shinji Kagawa  0.5243629217147827
Shinji Kagawa  0.546917200088501
Shinji Kagawa  0.5961862802505493
0Unknown  1.1471428871154785
0Unknown  1.196937918663025
0Unknown  1.200745940208435
0Unknown  1.199224829673767
0Unknown  1.1938974857330322
0Unknown  1.1797831058502197
0Unknown  1.1759233474731445
0Unknown  1.2050236463546753
0Unknown  1.198931097984314
0Unknown  1.2084436416625977
0Unknown  1.2311789989471436
0Unknown  1.2095328569412231
0Unknown  1.2180029153823853
0Unknown  1.2553637027740479
0Unknown  1.23139488697052
0Unknown  1.2860811948776245
0Unknown  1.2567747831344604
0Unknown  1.27733314037323
0Unknown  1.2971998453140259
0Unknown  1.2740466594696045
0Unknown  1.3280359506607056
0Unknown  1.3556292057037354
0Unknown  1.2974393367767334
0Unknown  1.381988525390625
0Unknown  1.3400447368621826
0Unknown  1.3670247793197632
0Unknown  1.2982465028762817
0Unknown  1.2948758602142334
0Unknown  1.2973041534423828
0Unknown  1.2898101806640625
0Unknown  1.2648670673370361
0Unknown  1.2893332242965698
0Unknown  1.279186487197876
0Unknown  1.2472201585769653
0Unknown  1.2449777126312256
0Unknown  1.2569679021835327
0Unknown  1.3036638498306274
0Unknown  1.2775137424468994
0Unknown  1.2824623584747314
0Unknown  1.3161582946777344
0Unknown  1.275862693786621
0Unknown  1.2979570627212524
0Unknown  1.2650532722473145
0Unknown  1.2425533533096313
0Unknown  1.3103786706924438
0Unknown  1.2631628513336182
0Unknown  1.3535734415054321
0Unknown  1.292794942855835
0Unknown  1.2784367799758911
0Unknown  1.2844229936599731
0Unknown  1.3222575187683105
0Unknown  0.934058666229248
0Unknown  0.9122346043586731
0Unknown  0.9394353032112122
0Unknown  0.9249307513237
0Unknown  0.9707850217819214
0Unknown  0.9289113879203796
0Unknown  0.9434537887573242
0Unknown  0.9440311193466187
0Unknown  0.9435308575630188
0Unknown  0.9643159508705139
0Unknown  0.928233802318573
0Unknown  0.9353509545326233
0Unknown  0.900050163269043
0Unknown  0.9527754783630371
0Unknown  0.9817777276039124
0Unknown  1.010622501373291
0Unknown  1.0198886394500732
0Unknown  0.976880669593811
0Unknown  1.3396625518798828
0Unknown  0.9890985488891602
0Unknown  0.9960650205612183
0Unknown  0.9901760816574097
0Unknown  1.4060555696487427
0Unknown  1.2538588047027588
0Unknown  1.2348939180374146
Tamara Mowry  0.6583441495895386
Tamara Mowry  0.6349061727523804
Tamara Mowry  0.7471032738685608
Tamara Mowry  0.7453776597976685
Tamara Mowry  0.7246230840682983
Tamara Mowry  0.6860607266426086
Tamara Mowry  0.7248824238777161
Tamara Mowry  0.6924018859863281
Tamara Mowry  0.7080976963043213
Tamara Mowry  0.7305541038513184
Tamara Mowry  0.7116721868515015
Tamara Mowry  0.7818515300750732
Tamara Mowry  0.7389234900474548
Tamara Mowry  0.7775006890296936
Tamara Mowry  0.7159475088119507
Tamara Mowry  0.7504268288612366
Tamara Mowry  0.7495276927947998
Tamara Mowry  0.7365091443061829
Tamara Mowry  0.7825095653533936
0Unknown  1.3347125053405762
0Unknown  1.2170848846435547
0Unknown  1.2518423795700073
0Unknown  1.2804245948791504
0Unknown  1.3002598285675049
0Unknown  1.335280418395996
0Unknown  1.3314138650894165
0Unknown  1.3259896039962769
0Unknown  1.3382421731948853
0Unknown  1.3382794857025146
0Unknown  1.351027488708496
0Unknown  1.3493072986602783
0Unknown  1.3391194343566895
0Unknown  1.3437491655349731
0Unknown  1.3558882474899292
0Unknown  1.3938651084899902
0Unknown  1.3741466999053955
0Unknown  1.3461024761199951
0Unknown  1.3423349857330322
0Unknown  1.295149803161621
0Unknown  1.2469381093978882
0Unknown  1.3050639629364014
Tamara Mowry  0.7250092625617981
Tamara Mowry  0.6800022125244141
Tamara Mowry  0.664611279964447
Tamara Mowry  0.6703887581825256
Tamara Mowry  0.7149563431739807
Tamara Mowry  0.7871045470237732
Tamara Mowry  0.6981989741325378
Tamara Mowry  0.6661180853843689
Tamara Mowry  0.6767582893371582
Tamara Mowry  0.7212544083595276
Tamara Mowry  0.7396418452262878
Tamara Mowry  0.6646618843078613
Tamara Mowry  0.6895917654037476
Tamara Mowry  0.6550708413124084
Tamara Mowry  0.7445973753929138
Tamara Mowry  0.7249574065208435
Tamara Mowry  0.6950424313545227
Tamara Mowry  0.7252428531646729
Tamara Mowry  0.6636046171188354
0Unknown  0.9740303158760071
vasili-berezutski  0.847652018070221
vasili-berezutski  0.8673129081726074
vasili-berezutski  0.8567150831222534
vasili-berezutski  0.8582270741462708
0Unknown  0.8889774680137634
vasili-berezutski  0.8100684881210327
0Unknown  0.8861156702041626
0Unknown  0.885290801525116
0Unknown  0.8819959759712219
vasili-berezutski  0.8462467193603516
vasili-berezutski  0.7990315556526184
vasili-berezutski  0.8556247353553772
vasili-berezutski  0.8337721824645996
0Unknown  0.8813753128051758
vasili-berezutski  0.8441100120544434
0Unknown  0.8872007727622986
vasili-berezutski  0.860283613204956
vasili-berezutski  0.8169236183166504
vasili-berezutski  0.8327621817588806
vasili-berezutski  0.8467702865600586
vasili-berezutski  0.8659766316413879
0Unknown  0.8844539523124695
0Unknown  0.8875654339790344
0Unknown  0.884465754032135
0Unknown  0.9316560626029968
0Unknown  0.9698878526687622
0Unknown  1.3430471420288086